In [1]:
# imports

# standard import of pandas
import pandas as pd

In [2]:
# getting dataframes

%store -r start_orders
%store -r start_order_process
%store -r start_intern_study
%store -r start_campaign


In [3]:
clean_orders = start_orders.copy()
clean_order_process = start_order_process.copy()
clean_intern_study = start_intern_study.copy()
clean_campaign = start_campaign.copy()

In [4]:
# function to make all the columns lower and snake case

def columns_format(data_frame):
	data_frame.columns = data_frame.columns.str.lower()
	data_frame.columns = data_frame.columns.str.replace(" ", "_")


In [5]:
columns_format(clean_orders)
columns_format(clean_order_process)
columns_format(clean_intern_study)
columns_format(clean_campaign)

In [6]:
# dropping columns in orders.csv

clean_orders.drop(["index","customer_id", "customer_name", "origin_channel","country/region", "city", "postal_code", "category", "sub-category", "product_id", "sales", "quantity", "discount", "profit"], axis=1, inplace=True)
clean_orders.drop_duplicates(inplace=True)
clean_orders.head(2)
clean_orders.duplicated().value_counts()

False    5009
Name: count, dtype: int64

In [7]:
# dropping row_id

clean_order_process.drop("row_id", axis=1, inplace=True)
clean_order_process.drop_duplicates(inplace=True)
clean_order_process.head()
clean_order_process.duplicated().value_counts()

False    3003
Name: count, dtype: int64

In [8]:
# dropping duplicates in intern data

clean_intern_study.drop_duplicates(inplace=True)
clean_intern_study.head(2)
clean_intern_study.duplicated().value_counts()

False    204
Name: count, dtype: int64

In [9]:
# dropping columns in campaign data

clean_campaign.drop(["customer_name"], axis=1, inplace=True)
clean_campaign.head(2)

,order_id,arrival_scan_date
0,CA-2019-109666,03/05/2019
1,CA-2019-138933,03/05/2019


In [10]:
first_merge = clean_orders.merge(clean_order_process, how="outer", on="order_id")

first_merge

,order_id,order_date_x,ship_mode_x,state,region,order_date_y,on_truck_scan_date,ship_mode_y
0,CA-2017-100006,7/9/2017,Standard Class,New York,East,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,Standard Class,California,West,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Standard Class,Florida,South,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,Standard Class,New York,East,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Standard Class,Arizona,West,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5005,US-2020-168802,3/11/2020,Standard Class,Washington,West,3/11/2020,12/11/2020,Standard Processing
5006,US-2020-169320,23/7/2020,Second Class,Indiana,Central,23/7/2020,29/07/2020,Standard Processing
5007,US-2020-169488,7/9/2020,First Class,Rhode Island,East,7/9/2020,09/09/2020,Express
5008,US-2020-169502,28/8/2020,Standard Class,Wisconsin,Central,28/8/2020,02/09/2020,Standard Processing


In [11]:
second_merge = first_merge.merge(clean_intern_study, how="outer", on="order_id")

second_merge

,order_id,order_date_x,ship_mode_x,state,region,order_date_y,on_truck_scan_date,ship_mode_y,ready_to_ship_date,pickup_date
0,CA-2017-100006,7/9/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,Standard Class,California,West,NaN,NaN,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Standard Class,Florida,South,NaN,NaN,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Standard Class,Arizona,West,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5005,US-2020-168802,3/11/2020,Standard Class,Washington,West,3/11/2020,12/11/2020,Standard Processing,09/11/2020,12/11/2020
5006,US-2020-169320,23/7/2020,Second Class,Indiana,Central,23/7/2020,29/07/2020,Standard Processing,NaN,NaN
5007,US-2020-169488,7/9/2020,First Class,Rhode Island,East,7/9/2020,09/09/2020,Express,NaN,NaN
5008,US-2020-169502,28/8/2020,Standard Class,Wisconsin,Central,28/8/2020,02/09/2020,Standard Processing,NaN,NaN


In [12]:
final_merge = second_merge.merge(clean_campaign, how="outer", on="order_id")
final_merge

,order_id,order_date_x,ship_mode_x,state,region,order_date_y,on_truck_scan_date,ship_mode_y,ready_to_ship_date,pickup_date,arrival_scan_date
0,CA-2017-100006,7/9/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,Standard Class,California,West,NaN,NaN,NaN,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Standard Class,Florida,South,NaN,NaN,NaN,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Standard Class,Arizona,West,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5005,US-2020-168802,3/11/2020,Standard Class,Washington,West,3/11/2020,12/11/2020,Standard Processing,09/11/2020,12/11/2020,NaN
5006,US-2020-169320,23/7/2020,Second Class,Indiana,Central,23/7/2020,29/07/2020,Standard Processing,NaN,NaN,NaN
5007,US-2020-169488,7/9/2020,First Class,Rhode Island,East,7/9/2020,09/09/2020,Express,NaN,NaN,NaN
5008,US-2020-169502,28/8/2020,Standard Class,Wisconsin,Central,28/8/2020,02/09/2020,Standard Processing,NaN,NaN,NaN


In [13]:
final_merge[final_merge['ship_mode_y'].isnull()]

,order_id,order_date_x,ship_mode_x,state,region,order_date_y,on_truck_scan_date,ship_mode_y,ready_to_ship_date,pickup_date,arrival_scan_date
0,CA-2017-100006,7/9/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,Standard Class,California,West,NaN,NaN,NaN,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Standard Class,Florida,South,NaN,NaN,NaN,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,Standard Class,New York,East,NaN,NaN,NaN,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Standard Class,Arizona,West,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4499,US-2018-167220,12/12/2018,Standard Class,Texas,Central,NaN,NaN,NaN,NaN,NaN,NaN
4500,US-2018-168704,13/4/2018,Standard Class,Texas,Central,NaN,NaN,NaN,NaN,NaN,NaN
4501,US-2018-168732,10/12/2018,Standard Class,Georgia,South,NaN,NaN,NaN,NaN,NaN,NaN
4502,US-2018-168914,21/5/2018,Standard Class,Illinois,Central,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
## fill null values from ship_mode:y with values from ship_mode_x
final_merge['ship_mode_y'] = final_merge['ship_mode_y'].fillna(final_merge['ship_mode_x'])

## check there are no null values in ship_mode_y
final_merge[final_merge['ship_mode_y'].isnull()]

final_merge.drop("ship_mode_x", axis=1, inplace=True)
final_merge.rename(columns={"ship_mode_y": "ship_mode"}, inplace=True)
final_merge.head()

,order_id,order_date_x,state,region,order_date_y,on_truck_scan_date,ship_mode,ready_to_ship_date,pickup_date,arrival_scan_date
0,CA-2017-100006,7/9/2017,New York,East,NaN,NaN,Standard Class,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,California,West,NaN,NaN,Standard Class,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Florida,South,NaN,NaN,Standard Class,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,New York,East,NaN,NaN,Standard Class,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Arizona,West,NaN,NaN,Standard Class,NaN,NaN,NaN


In [16]:
final_merge.insert(2, "order_date", final_merge["order_date_x"])
final_merge.drop(["order_date_x", "order_date_y"], axis=1, inplace=True)
final_merge.head(15)

,order_id,order_date,state,region,on_truck_scan_date,ship_mode,ready_to_ship_date,pickup_date,arrival_scan_date
0,CA-2017-100006,7/9/2017,New York,East,NaN,Standard Class,NaN,NaN,NaN
1,CA-2017-100090,8/7/2017,California,West,NaN,Standard Class,NaN,NaN,NaN
2,CA-2017-100293,14/3/2017,Florida,South,NaN,Standard Class,NaN,NaN,NaN
3,CA-2017-100328,28/1/2017,New York,East,NaN,Standard Class,NaN,NaN,NaN
4,CA-2017-100363,8/4/2017,Arizona,West,NaN,Standard Class,NaN,NaN,NaN
5,CA-2017-100391,25/5/2017,New York,East,NaN,Standard Class,NaN,NaN,NaN
6,CA-2017-100678,18/4/2017,Texas,Central,NaN,Standard Class,NaN,NaN,NaN
7,CA-2017-100706,16/12/2017,Virginia,South,NaN,Second Class,NaN,NaN,NaN
8,CA-2017-100762,24/11/2017,Michigan,Central,NaN,Standard Class,NaN,NaN,NaN
9,CA-2017-100860,26/3/2017,California,West,NaN,Second Class,NaN,NaN,NaN


In [17]:
#final_merge.insert(2, "ship_mode", final_merge["ship_mode_y"])
#final_merge.drop(["ship_mode_x", "ship_mode_y"], axis=1, inplace=True)
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5010 entries, 0 to 5009
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            5010 non-null   object
 1   order_date          5010 non-null   object
 2   state               5010 non-null   object
 3   region              5010 non-null   object
 4   on_truck_scan_date  3003 non-null   object
 5   ship_mode           5010 non-null   object
 6   ready_to_ship_date  204 non-null    object
 7   pickup_date         204 non-null    object
 8   arrival_scan_date   333 non-null    object
dtypes: object(9)
memory usage: 352.4+ KB


In [18]:
final_merge['order_date'] = pd.to_datetime(final_merge["order_date"], format="%d/%m/%Y")
final_merge['ready_to_ship_date'] = pd.to_datetime(final_merge["ready_to_ship_date"], format="%d/%m/%Y")
final_merge['pickup_date'] = pd.to_datetime(final_merge["pickup_date"], format="%d/%m/%Y")
final_merge['arrival_scan_date'] = pd.to_datetime(final_merge["arrival_scan_date"], format="%d/%m/%Y")
final_merge['on_truck_scan_date'] = pd.to_datetime(final_merge["on_truck_scan_date"], format="%d/%m/%Y")

In [19]:
final_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5010 entries, 0 to 5009
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            5010 non-null   object        
 1   order_date          5010 non-null   datetime64[ns]
 2   state               5010 non-null   object        
 3   region              5010 non-null   object        
 4   on_truck_scan_date  3003 non-null   datetime64[ns]
 5   ship_mode           5010 non-null   object        
 6   ready_to_ship_date  204 non-null    datetime64[ns]
 7   pickup_date         204 non-null    datetime64[ns]
 8   arrival_scan_date   333 non-null    datetime64[ns]
dtypes: datetime64[ns](5), object(4)
memory usage: 352.4+ KB


In [27]:
## fill null values from ship_mode:y with values from ship_mode_x
final_merge['pickup_date'] = final_merge['pickup_date'].fillna(final_merge['on_truck_scan_date'])
final_merge.drop("on_truck_scan_date", axis=1, inplace=True)
final_merge[final_merge['pickup_date'].isnull()]


,order_id,order_date,state,region,ship_mode,ready_to_ship_date,pickup_date,arrival_scan_date
4980,US-2020-162558,2020-10-02,Tennessee,South,Express,NaT,2020-10-05,NaT
4981,US-2020-162670,2020-12-23,Arkansas,South,Standard Processing,NaT,2020-12-30,NaT
4982,US-2020-163195,2020-02-17,Louisiana,South,Standard Processing,NaT,2020-02-24,2020-03-02
4983,US-2020-163300,2020-09-15,California,West,Standard Processing,NaT,2020-09-23,NaT
4984,US-2020-163657,2020-09-02,California,West,Standard Processing,NaT,2020-09-09,NaT
4985,US-2020-163790,2020-11-02,California,West,Standard Processing,2020-11-04,2020-11-06,NaT
4986,US-2020-164056,2020-04-29,Ohio,East,Standard Processing,NaT,2020-05-06,NaT
4987,US-2020-164147,2020-02-02,Ohio,East,Express,NaT,2020-02-07,NaT
4988,US-2020-165344,2020-11-13,Ohio,East,Express,2020-11-16,2020-11-16,NaT
4989,US-2020-165358,2020-07-18,Pennsylvania,East,Standard Processing,NaT,2020-07-22,NaT


In [28]:
%store final_merge

Stored 'final_merge' (DataFrame)
